
# Parallel Chain Example - Payment Notification

## Goal
- Input: Payment object
- Generate disclaimer (English & French)
- If payment completed:
  - Send Email
  - Send SMS

👉 Email and SMS should run in **parallel**


In [1]:

payment = {
    "amount": 1000,
    "payer_name": "Subbu",
    "payment_method": "Credit Card",
    "status": "completed"
}
print(payment)


{'amount': 1000, 'payer_name': 'Subbu', 'payment_method': 'Credit Card', 'status': 'completed'}



## Flow

Input → Disclaimer Chain → Condition (completed?)
                                ↓
                         Parallel Execution
                         /               \
                    Send Email         Send SMS


In [2]:

from pydantic import BaseModel, Field

class Disclaimer(BaseModel):
    english: str
    french: str


In [3]:

from langchain.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=Disclaimer)


In [4]:

from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "Generate payment disclaimer in English and French"),
    ("human", "Payment details: {payment}\n{format_instructions}")
])

prompt = prompt.partial(format_instructions=parser.get_format_instructions())


In [5]:

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-3-flash-preview",
    convert_system_message_to_human=True
)


d:\LangChain\langchain\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:

# Disclaimer Chain
disclaimer_chain = prompt | llm | parser



## Parallel Tasks (Email & SMS)


In [7]:

from langchain_core.runnables import RunnableLambda, RunnableParallel

def send_email(data):
    return f"Email sent to {data['payer_name']} for {data['amount']}"

def send_sms(data):
    return f"SMS sent to {data['payer_name']} for {data['amount']}"

email_runnable = RunnableLambda(send_email)
sms_runnable = RunnableLambda(send_sms)

parallel_tasks = RunnableParallel({
    "email": email_runnable,
    "sms": sms_runnable
})


In [8]:

# Combine disclaimer + parallel tasks

def process(payment):
    disclaimer = disclaimer_chain.invoke({"payment": payment})

    result = {
        "disclaimer": disclaimer
    }

    if payment["status"] == "completed":
        notifications = parallel_tasks.invoke(payment)
        result["notifications"] = notifications

    return result


In [9]:

result = process(payment)

print(result)


{'disclaimer': Disclaimer(english='This confirms that a payment of 1000 has been successfully completed by Subbu using a Credit Card. This transaction is subject to the terms and conditions of the service agreement. Please retain this notice for your records.', french="Ceci confirme qu'un paiement de 1000 a été complété avec succès par Subbu via Carte de Crédit. Cette transaction est soumise aux conditions générales du contrat de service. Veuillez conserver cet avis pour vos dossiers."), 'notifications': {'email': 'Email sent to Subbu for 1000', 'sms': 'SMS sent to Subbu for 1000'}}
